**Design your own Streamlit Code** and play the following code for the ***Movie Recommendation Agent***



In [1]:
!pip install langgraph langchain-core langchain-google-genai requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 10.1 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelang

In [2]:
# --- 2. Imports and Setup ---
import requests
import random
import re
from typing import TypedDict, Optional, Literal, List
from langgraph.graph import StateGraph, END
from langchain_core.tools import Tool
from langchain_google_genai import ChatGoogleGenerativeAI

In [3]:
# --- 3. Define MovieState ---
class MovieState(TypedDict):
    year: Optional[str]
    movie: Optional[str]
    next_action: Optional[Literal["continue", "exit"]]
    history: List[str]
    count: int
    genre: Optional[str]

In [5]:
#4. Movie Tool Function ---
def movie_tool(year: str, count: int = 5, genre: str = None) -> dict:
    url = "https://raw.githubusercontent.com/prust/wikipedia-movie-data/master/movies.json"
    movies = requests.get(url).json()

    # Filter by year
    year_int = int(year)
    filtered = [m for m in movies if m["year"] == year_int]

    # Filter by genre if provided
    if genre:
        filtered = [m for m in filtered if genre.lower() in [g.lower() for g in m.get("genres", [])]]

    # Random selection
    if not filtered:
        return {"movies": [], "llm_recommendations": "No movies found for that year/genre."}

    chosen_movies = random.sample(filtered, min(count, len(filtered)))
    movie_list = [f"{m['title']} ({m['year']})" for m in chosen_movies]

    # --- Gemini LLM Recommendation ---
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0.7,
        google_api_key=""  # 🔑 Replace with your key
    )

    prompt = f"""
    Here are some movies: {', '.join([m['title'] for m in chosen_movies])}.
    For each movie:
    - Give a short (max 2 sentences) recommendation or description.
    - Style depends on genre: comedy = funny, horror = spooky, drama = emotional, action = thrilling, romance = heartfelt.
    - Add a rating out of 10.
    - Mention release year if available.

    Then:
    - Identify the best and top-rated movie among them.
    - Provide a one-line comparison of why it stands out.
    - Include a link to the rating source for the best movie.

    Return the results as a Markdown table with columns:
    Title | Genre | Year | Recommendation | Rating/10 |
    """

    llm_recommendations = llm.invoke(prompt).content
    return {"movies": movie_list, "llm_recommendations": llm_recommendations}



In [6]:
# --- 5. Create Tool Wrapper ---
movie_tool_agent = Tool(
    name="MovieTool",
    description="Returns random movies from a given year and genre + Gemini recommendations",
    func=movie_tool
)

In [7]:
# --- 6. Graph Node Functions ---
def ask_user_query(state: MovieState) -> MovieState:
    query = input("🎥 Enter your movie request (e.g., '5 action movies from 2015'): ").strip()
    year_match = re.search(r"\b(19|20)\d{2}\b", query)
    count_match = re.search(r"\b\d+\b", query)
    genre_match = re.search(r"(action|comedy|drama|horror|romance|thriller|animation)", query, re.I)

    state["year"] = year_match.group(0) if year_match else None
    state["count"] = int(count_match.group(0)) if count_match else 5
    state["genre"] = genre_match.group(0).lower() if genre_match else None
    return state


def suggest_movie(state: MovieState) -> MovieState:
    result = movie_tool_agent.func(
        year=state["year"],
        count=state["count"],
        genre=state["genre"]
    )
    state["movie"] = ", ".join(result["movies"])
    state["history"].extend(result["movies"])

    print("\n🎬 Suggested movies:", state["movie"])
    print("\n🤖 Gemini Recommendations:\n", result["llm_recommendations"])
    return state


def should_continue(state: MovieState) -> MovieState:
    state["next_action"] = input("\nType 'continue' for another recommendation or 'exit' to stop: ").strip().lower()
    return state


def decide_route(state: MovieState) -> str:
    return "ask_user_query" if state["next_action"] == "continue" else END


In [8]:
# --- 7. Create LangGraph Workflow ---
graph = StateGraph(MovieState)
graph.add_node("ask_user_query", ask_user_query)
graph.add_node("suggest_movie", suggest_movie)
graph.add_node("should_continue", should_continue)
graph.set_entry_point("ask_user_query")
graph.add_edge("ask_user_query", "suggest_movie")
graph.add_edge("suggest_movie", "should_continue")
graph.add_conditional_edges("should_continue", decide_route)

app = graph.compile()


In [ ]:
# --- 8. Run the CineGraph Agent ---
print("🎬 Welcome to CineGraph (Colab Version)!")
print("Discover random movies with AI recommendations from Gemini.\n")

initial_state: MovieState = {"year": None, "movie": None, "next_action": "continue", "history": [], "count": 5, "genre": None}

app.invoke(initial_state)

🎬 Welcome to CineGraph (Colab Version)!
Discover random movies with AI recommendations from Gemini.

🎥 Enter your movie request (e.g., '5 action movies from 2015'): 5 action movies from 2015

🎬 Suggested movies: Hot Pursuit (2015), Paul Blart: Mall Cop 2 (2015), Bad Asses on the Bayou (2015), Extraction (2015), Kingsman: The Secret Service (2015)

🤖 Gemini Recommendations:
 Here are the details for each movie:

| Title | Genre | Year | Recommendation | Rating/10 |
| :-------------------------- | :------------------ | :--- | :-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

KeyboardInterrupt: Interrupted by user